In [0]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *

In [0]:
#dbutils.secrets.listScopes()

In [0]:
#dbutils.secrets.list(scope="databricksscope")

In [0]:
spark.conf.set("fs.azure.account.key.optumsadllssa.dfs.core.windows.net",dbutils.secrets.get(scope="databricksscope",key="adlskey"))

In [0]:
display(dbutils.fs.ls("abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/Claims.csv,Claims.csv,5766,1716728851000
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/Patient_records.csv,Patient_records.csv,5110,1716718820000
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/disease.csv,disease.csv,1489,1716718820000
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/group.csv,group.csv,4390,1716718820000
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/hospital.csv,hospital.csv,1528,1716728950000
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/subgroup.csv,subgroup.csv,561,1716718820000
abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/subscriber.csv,subscriber.csv,12061,1716718820000


In [0]:
data = spark.read.csv('abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/Patient_records.csv',header=True)#inferSchema=True)

In [0]:
data.show(5,False)

+----------+------------+--------------+------------------+--------------+--------------+------------+-----------+
|Patient_id|Patient_name|patient_gender|patient_birth_date|patient_phone |disease_name  |city        |hospital_id|
+----------+------------+--------------+------------------+--------------+--------------+------------+-----------+
|187158    |Harbir      |Female        |1924-06-30        |+91 0112009318|Galactosemia  |Rourkela    |H1001      |
|112766    |Brahmdev    |Female        |1948-12-20        |+91 1727749552|Bladder cancer|Tiruvottiyur|H1016      |
|199252    |Ujjawal     |Male          |1980-04-16        |+91 8547451606|Kidney cancer |Berhampur   |H1009      |
|133424    |Ballari     |Female        |1969-09-25        |+91 0106026841|Suicide       |Bihar Sharif|H1017      |
|172579    |Devnath     |Female        |1946-05-01        |+91 1868774631|Food allergy  |Bidhannagar |H1019      |
+----------+------------+--------------+------------------+--------------+------

In [0]:
data.count()

70

In [0]:
data.select([count(when(isnan(c)|col(c).isNull(),c)).alias(c) for c in data.columns]).show()

+----------+------------+--------------+------------------+-------------+------------+----+-----------+
|Patient_id|Patient_name|patient_gender|patient_birth_date|patient_phone|disease_name|city|hospital_id|
+----------+------------+--------------+------------------+-------------+------------+----+-----------+
|         0|          17|             0|                 0|            2|           0|   0|          0|
+----------+------------+--------------+------------------+-------------+------------+----+-----------+



In [0]:
data.groupby(data.columns).count().where("count>1").show()

+----------+------------+--------------+------------------+-------------+------------+----+-----------+-----+
|Patient_id|Patient_name|patient_gender|patient_birth_date|patient_phone|disease_name|city|hospital_id|count|
+----------+------------+--------------+------------------+-------------+------------+----+-----------+-----+
+----------+------------+--------------+------------------+-------------+------------+----+-----------+-----+



In [0]:
data = data.dropDuplicates()

In [0]:
data = data.fillna({'Patient_name':'Guest/NA'})

In [0]:
data = data.withColumn("Patient_age",(months_between(current_date(),col('patient_birth_date'))/12).cast("int"))

In [0]:
data.show(5,False)

+----------+------------+--------------+------------------+--------------+--------------+------------+-----------+-----------+
|Patient_id|Patient_name|patient_gender|patient_birth_date|patient_phone |disease_name  |city        |hospital_id|Patient_age|
+----------+------------+--------------+------------------+--------------+--------------+------------+-----------+-----------+
|133424    |Ballari     |Female        |1969-09-25        |+91 0106026841|Suicide       |Bihar Sharif|H1017      |54         |
|112766    |Brahmdev    |Female        |1948-12-20        |+91 1727749552|Bladder cancer|Tiruvottiyur|H1016      |75         |
|171320    |Atasi       |Male          |1967-10-02        |+91 9747336855|Whiplash      |Amravati    |H1013      |56         |
|187158    |Harbir      |Female        |1924-06-30        |+91 0112009318|Galactosemia  |Rourkela    |H1001      |99         |
|199252    |Ujjawal     |Male          |1980-04-16        |+91 8547451606|Kidney cancer |Berhampur   |H1009    

In [0]:
data = data.drop(col("patient_birth_date"))

In [0]:
data = data.fillna({"patient_phone":"NA"})

In [0]:
data.select([count(when(isnan(c)|col(c).isNull(),c)).alias(c) for c in data.columns]).show()

+----------+------------+--------------+-------------+------------+----+-----------+-----------+
|Patient_id|Patient_name|patient_gender|patient_phone|disease_name|city|hospital_id|Patient_age|
+----------+------------+--------------+-------------+------------+----+-----------+-----------+
|         0|           0|             0|            0|           0|   0|          0|          0|
+----------+------------+--------------+-------------+------------+----+-----------+-----------+



In [0]:

output_container_path = "abfss://optumstagingdata@optumsadllssa.dfs.core.windows.net"
output_blob_folder = "optumstagingdata"
data.coalesce(1).write.format("csv").mode("overwrite").option("header","true").format("com.databricks.spark.csv").save(output_blob_folder)
files = dbutils.fs.ls(output_blob_folder)
output_file = [x for x in files if x.name.startswith("part-")]
dbutils.fs.mv(output_file[0].path,"%s/patientdatastaging.csv"%output_container_path)

True